In [5]:
!pip install scikit-learn


You should consider upgrading via the 'C:\Users\PAYAL\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [3]:
df = pd.read_csv('../data/hiring_data.csv')


Offer Acceptance Prediction Model 

In [9]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

X = df[['experience_years', 'skills_score', 'communication_score', 'salary_offered']]
y = df['offer_accepted']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(
        max_iter=2000,
        class_weight='balanced'
    ))
])

pipeline.fit(X_train, y_train)

# probability of acceptance
y_prob = pipeline.predict_proba(X_test)[:, 1]

# lower threshold to improve recall
threshold = 0.45
y_pred_custom = (y_prob >= threshold).astype(int)

print(classification_report(y_test, y_pred_custom))



              precision    recall  f1-score   support

           0       0.62      0.29      0.39       104
           1       0.51      0.81      0.63        96

    accuracy                           0.54       200
   macro avg       0.57      0.55      0.51       200
weighted avg       0.57      0.54      0.51       200



In [10]:
pd.DataFrame({
    'actual': y_test.values,
    'acceptance_probability': y_prob,
    'predicted': y_pred_custom
}).head(10)


,actual,acceptance_probability,predicted
0,1,0.581217,1
1,0,0.495393,1
2,1,0.550499,1
3,0,0.600195,1
4,1,0.614124,1
5,1,0.589040,1
6,1,0.399100,0
7,0,0.447366,0
8,0,0.426987,0
9,0,0.545810,1


Early Attrition prediction model 

In [11]:
X2 = df[['experience_years', 'communication_score', 'salary_offered']]
y2 = df['left_within_6_months']

X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2, test_size=0.2, random_state=42
)

rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight='balanced'
)

rf.fit(X2_train, y2_train)

# get probability of attrition (class = 1)
y_prob = rf.predict_proba(X2_test)[:, 1]

# custom threshold (lower than 0.5)
threshold = 0.35
y_pred_custom = (y_prob >= threshold).astype(int)

print(classification_report(y2_test, y_pred_custom))

              precision    recall  f1-score   support

           0       0.79      0.74      0.77       154
           1       0.29      0.35      0.31        46

    accuracy                           0.65       200
   macro avg       0.54      0.54      0.54       200
weighted avg       0.68      0.65      0.66       200



In [7]:
pd.DataFrame({
    'actual': y2_test.values,
    'risk_probability': y_prob,
    'predicted': y_pred_custom
}).head(10)


,actual,risk_probability,predicted
0,0,0.305,0
1,1,0.425,1
2,0,0.365,1
3,0,0.085,0
4,0,0.270,0
5,1,0.700,1
6,0,0.000,0
7,0,0.640,1
8,0,0.160,0
9,1,0.310,0


In [40]:
pd.Series(rf.feature_importances_, index=X2.columns).sort_values()


communication_score    0.291137
experience_years       0.304486
salary_offered         0.404377
dtype: float64